In [1]:
# Replace 'path/to/requirements.txt' with the actual path to your requirements.txt file
%pip install -r /home/aalab/Desktop/tripoFT/TripoSR/requirements.txt

  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-j_4dxwc2
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-j_4dxwc2
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3aef8afa5f21b113afc4f4ea148baee850cbd472
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [21]:
import sys
import os

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR
from omegaconf import OmegaConf


In [22]:
import sys
import os

# Add the TripoSR directory to the Python path
sys.path.append(os.path.abspath('TripoSR'))

# Import the TSR class
from tsr.system import TSR
from omegaconf import OmegaConf


In [26]:
# Load the configuration file
config_path = 'TripoSR/config.yaml'
cfg = OmegaConf.load(config_path)

# Manually add missing keys
cfg.tokenizer.num_channels = 3  # Replace with the correct value
# Add any other required keys here

# Initialize the TSR model with the loaded and updated configuration
model = TSR(cfg=cfg)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


InterpolationKeyError: Interpolation key 'tokenizer.num_channels' not found
    full_key: in_channels
    reference_type=Dict[Any, Any]
    object_type=Config

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader

class CombinedTensorDataset(Dataset):
    def __init__(self, data_dir):
        self.tensor_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.pt')])
        self.data_dir = data_dir

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_path = os.path.join(self.data_dir, self.tensor_files[idx])
        combined_tensor = torch.load(tensor_path)
        vertices, faces = combined_tensor
        return vertices, faces

def get_dataloaders(train_dir, val_dir, test_dir, batch_size=32):
    train_dataset = CombinedTensorDataset(train_dir)
    val_dataset = CombinedTensorDataset(val_dir)
    test_dataset = CombinedTensorDataset(test_dir)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

In [19]:
# Define paths to your data directories
train_data_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_data_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_data_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# Load the data
train_loader, val_loader, test_loader = get_dataloaders(train_data_dir, val_data_dir, test_data_dir)

In [20]:
model = TSR(pretrained=True)  # Assuming the TSR class has a pretrained argument

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = torch.nn.MSELoss()  # Use appropriate loss for your task
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


ValueError: Cannot merge with a None config
    full_key: 
    object_type=Config